In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math
from math import sin, cos, radians

from lib2d import Objects2D as ob2d
from lib2d import Transform2D as tf2d

### Código base usado em aula

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(800, 800, "T1 - CG", None, None)
glfw.make_context_current(window)

vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")
    
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")
    
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Criação dos objetos

Cada objeto é composto por formas. As formas são posicionadas dentro do objeto pensando nas coordenadas locais do objeto. Cada forma possui uma primitiva e uma cor.

Após posicionar todas as formas, é possível manipular o objeto como um todo alterando seu t_x, t_y, e_x, e_y e r

In [3]:
# Forms
# Chão
obj_floor = ob2d.Object2D()
form_floor = ob2d.makeRectangle((0, 0), 2, 0.2, 0)
obj_floor.addForm(form_floor, GL_TRIANGLE_STRIP, (128/255, 227/255, 77/255, 1.0))
obj_floor.t_y = -1

# Catavento_cima
obj_catavento = ob2d.Object2D()
form_catav_centro = ob2d.makeCircle((0, 0), 0.05, 30)
form_catav_pa1 = [(0.05,0),(0.15,-0.05),(0.15,0.05),(0.25,0)]
form_catav_pa2 = [(-0.05,0),(-0.15,-0.05),(-0.15,0.05),(-0.25,0)]
form_catav_pa3 = [(0,0.05),(-0.05,0.15),(0.05,0.15),(0,0.25)]
form_catav_pa4 = [(0,-0.05),(-0.05,-0.15),(0.05,-0.15),(0,-0.25)]
obj_catavento.addForm(form_catav_centro, GL_TRIANGLE_FAN, (1.0, 1.0, 0, 1.0))
obj_catavento.addForm(form_catav_pa1, GL_TRIANGLE_STRIP, (0, 1.0, 0, 1.0))
obj_catavento.addForm(form_catav_pa2, GL_TRIANGLE_STRIP, (0, 1.0, 0, 1.0))
obj_catavento.addForm(form_catav_pa3, GL_TRIANGLE_STRIP, (0, 0, 1.0, 1.0))
obj_catavento.addForm(form_catav_pa4, GL_TRIANGLE_STRIP, (0, 0, 1.0, 1.0))
obj_catavento.t_x = 0.3
obj_catavento.t_y = -0.7
obj_catavento.e_x = 0.4
obj_catavento.e_y = 0.4

# Catavento_haste
obj_catav_haste = ob2d.Object2D()
form_catav_haste = ob2d.makeRectangle((0,-0.225), 0.02, 0.45, 0)
obj_catav_haste.addForm(form_catav_haste, GL_TRIANGLE_STRIP, (0.5, 0.5, 0, 1.0))
obj_catav_haste.t_x = 0.3
obj_catav_haste.t_y = -0.81
obj_catav_haste.e_x = 0.4
obj_catav_haste.e_y = 0.4

# Player
obj_player = ob2d.Object2D()
form_player_body = ob2d.makeRectangle((0, 0), 0.1, 0.2, 0)
form_player_head = ob2d.makeCircle((0, 0.15), 0.05, 30)
form_player_hat = ob2d.makeEquilateralTriangle((0, 0.235), 0.1, 0)
form_player_leftarm = [(0.05, 0.08), (0.09, 0.01)]
form_player_rightarm = [(-0.05, 0.08), (-0.09, 0.01)]
obj_player.addForm(form_player_body, GL_TRIANGLE_STRIP, (249/255, 108/255, 47/255, 1.0))
obj_player.addForm(form_player_head, GL_TRIANGLE_FAN, (47/255, 249/255, 173/255, 1.0))
obj_player.addForm(form_player_hat, GL_TRIANGLES, (255/255, 248/255, 82/255, 1.0))
obj_player.addForm(form_player_leftarm, GL_LINES, (0.0,0.0,0.0,1.0))
obj_player.addForm(form_player_rightarm, GL_LINES, (0.0,0.0,0.0,1.0))
obj_player.t_x = -0.8
obj_player.t_y = -0.705

# Arvore
obj_tree = ob2d.Object2D()
form_tree_log = ob2d.makeRectangle((0,0), 0.1, 0.4, 0)
form_tree_leaves = ob2d.makeCircle((0, 0.3), 0.2, 30)
obj_tree.addForm(form_tree_log, GL_TRIANGLE_STRIP, (231/255,164/255,61/255))
obj_tree.addForm(form_tree_leaves, GL_TRIANGLE_FAN, (126/255,197/255,0))
obj_tree.setPivotPoint((0,-0.2))
obj_tree.t_x = 0.6
obj_tree.t_y = -0.9
obj_tree.e_x = 0.6
obj_tree.e_y = 0.6

# Maçã
obj_apple = ob2d.Object2D()
form_apple_right_half = [(0, 0.43),(0.14,0.54),(0.36,0.53),(0.49,0.41),(0.51,0.27),(0.49,0.12),(0.44,-0.03),
                  (0.33,-0.15),(0.2,-0.2),(0.05,-0.19),(0,-0.15)]
form_apple_left_half = [(0, 0.43),(-0.14,0.54),(-0.36,0.53),(-0.49,0.41),(-0.51,0.27),(-0.49,0.12),(-0.44,-0.03),
                  (-0.33,-0.15),(-0.2,-0.2),(-0.05,-0.19),(0,-0.15)]
form_apple_leave = [(0,0.51),(-0.02,0.59),(-0.02,0.65),(0,0.7),(0.05,0.75),(0.11,0.78),(0.12,0.73),(0.12,0.67),(0.1,0.63),(0.05,0.56)]
obj_apple.addForm(form_apple_right_half, GL_TRIANGLE_FAN, (0.9, 0.0, 0.0, 1.0))
obj_apple.addForm(form_apple_left_half, GL_TRIANGLE_FAN, (0.9, 0.0, 0.0, 1.0))
obj_apple.addForm(form_apple_leave, GL_TRIANGLE_FAN, (0, 0.9, 0.3, 1.0))
obj_apple.t_x = -0.2
obj_apple.t_y = -0.8
obj_apple.e_x = 0.1
obj_apple.e_y = 0.1

# Sol
obj_sun = ob2d.Object2D()
form_sun_core = ob2d.makeCircle((0,0), 0.2, 30)
obj_sun.addForm(form_sun_core, GL_TRIANGLE_FAN, (255/255,219/255,0/255))
for i in range(16):
    form_sun_ray = []
    for v in ob2d.makeRectangle((0.275, 0), 0.1, 0.01, 0):
        form_sun_ray.append(ob2d.rotate_point(v, 360/16*i))
    obj_sun.addForm(form_sun_ray, GL_TRIANGLE_STRIP, (255/255,219/255,0/255))
obj_sun.t_x = -1
obj_sun.t_y = 1
obj_sun.e_x = 1.5
obj_sun.e_y = 1.5

# Arco-íris
obj_rainbow = ob2d.Object2D()
form_rainbow_red = ob2d.makeCircle((0,0), 2*math.sqrt(2)+0.30, 60)
form_rainbow_orange = ob2d.makeCircle((0,0), 2*math.sqrt(2)+0.25, 60)
form_rainbow_yellow = ob2d.makeCircle((0,0), 2*math.sqrt(2)+0.20, 60)
form_rainbow_green = ob2d.makeCircle((0,0), 2*math.sqrt(2)+0.15, 60)
form_rainbow_blue = ob2d.makeCircle((0,0), 2*math.sqrt(2)+0.10, 60)
form_rainbow_indigo = ob2d.makeCircle((0,0), 2*math.sqrt(2)+0.05, 60)
form_rainbow_violet = ob2d.makeCircle((0,0), 2*math.sqrt(2), 60)
form_rainbow_sky = ob2d.makeCircle((0,0), 2*math.sqrt(2)-0.05, 60)
obj_rainbow.addForm(form_rainbow_red, GL_TRIANGLE_FAN, (236/255, 0/255, 0/255, 1))
obj_rainbow.addForm(form_rainbow_orange, GL_TRIANGLE_FAN, (236/255, 143/255, 0/255, 1))
obj_rainbow.addForm(form_rainbow_yellow, GL_TRIANGLE_FAN, (234/255, 236/255, 0/255, 1))
obj_rainbow.addForm(form_rainbow_green, GL_TRIANGLE_FAN, (67/255, 236/255, 0/255, 1))
obj_rainbow.addForm(form_rainbow_blue, GL_TRIANGLE_FAN, (0/255, 16/255, 236/255, 1))
obj_rainbow.addForm(form_rainbow_indigo, GL_TRIANGLE_FAN, (127/255, 0/255, 236/255, 1))
obj_rainbow.addForm(form_rainbow_violet, GL_TRIANGLE_FAN, (186/255, 0/255, 236/255, 1))
obj_rainbow.addForm(form_rainbow_sky, GL_TRIANGLE_FAN, (207/255, 243/255, 251/255, 1.0))
obj_rainbow.t_x = 1
obj_rainbow.t_y = -2.25
    
    


# SERIALIZAÇÃO
objects = [obj_rainbow, obj_catav_haste, obj_catavento, obj_player, obj_tree, obj_floor, obj_apple, obj_sun]

ser_arrays, offsets = ob2d.serializeObjects2D(objects)

### Preparando os dados e enviando para a GPU

In [4]:
# preparando espaço para N vértices usando 2 coordenadas (x,y)
vertices = np.zeros(len(ser_arrays), [("position", np.float32, 2)])

vertices['position'] = ser_arrays

# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


# Pegando referências para as variáveis do programa
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

loc_color = glGetUniformLocation(program, "color")

### Variáveis auxiliares para a lógica da maça, pulo do player e rotação do sol

In [5]:
apple_frame = 0
apple_ancor_point = obj_apple.t_y

plr_yVel = 0
plr_ancor_point = obj_player.t_y
plr_grounded = True

sun_r = 0

### Mapeado o teclado para controlar o player, catavendo e árvore

In [6]:
# Mapeando teclas para eventos

def key_event(window,key,scancode,action,mods):
    
    global plr_yVel, plr_grounded
    
    if key == 265:
        if(obj_tree.e_x < 1.2):
            obj_tree.e_x += 0.01    #cima
            obj_tree.e_y += 0.01
    if key == 264:
        if(obj_tree.e_x > 0.3):
            obj_tree.e_x -= 0.01    #baixo
            obj_tree.e_y -= 0.01
    if key == 263: obj_catavento.r += 11.25    #esquerda
    if key == 262: obj_catavento.r -= 11.25    #direita
        
    if key == 65: obj_player.t_x -= 0.01        # A
    if key == 68: obj_player.t_x += 0.01        # D
    if (key == 87 and plr_grounded): 
        plr_yVel = 0.04                # W
        plr_grounded = False
        
        
glfw.set_key_callback(window,key_event)

### Lógica da maça, pulo do player e sol. Desenho dos objetos

In [7]:
glfw.show_window(window)

while not glfw.window_should_close(window):
    
    # APPLE LOGIC-----------------------------------------------
    apple_frame += 1
    apple_frame %= 100
    apple_movement_amplitude = 0.02
    # apple's y follows a cossine function
    obj_apple.t_y = apple_ancor_point + math.cos(apple_frame/100 * (2*math.pi)) * apple_movement_amplitude
    
    # if the player touches the apple, move it off screen
    if(obj_player.t_x > obj_apple.t_x - 0.51 * obj_apple.e_x and obj_player.t_x < obj_apple.t_x + 0.51 * obj_apple.e_x):
        obj_apple.t_x = 2
    
    # PLAYER JUMP LOGIC-----------------------------------------
    if(obj_player.t_y > plr_ancor_point):
        plr_yVel -= 0.003
    elif(plr_yVel < 0):
        plr_yVel = 0
        plr_grounded = True
    
    obj_player.t_y += plr_yVel
    
    # SUN ROTATION LOGIC---------------------------------------
    sun_r -= 0.055
    sun_r %= 360
    obj_sun.r = sun_r
    
    # DRAWING ON SCREEN---------------------------------------
    glfw.poll_events() 

    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(207/255, 243/255, 251/255, 1.0)
    
    loc_mat = glGetUniformLocation(program, "mat_transformation")
    
    for i in range(len(objects)):
        ob2d.draw2dObject(objects[i], offsets[i], loc_mat, loc_color)


    glfw.swap_buffers(window)

glfw.terminate()